<a href="https://colab.research.google.com/github/Nayanatharapmc/Amazon_Sentiment_Analysis/blob/main/AspectBasedSentimentAnalysisLLama3Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# Getting Aspects of the Reviews using Llama3
***

In [1]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
GROQ_API_TOKEN = userdata.get('groq')

os.environ["GROQ_API_KEY"] = GROQ_API_TOKEN

In [3]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)



def llama3_70b(prompt, temperature=0.0, input_print=True):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="llama3-70b-8192",
      temperature=temperature,
  )

  return (chat_completion.choices[0].message.content)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
apple_df = pd.read_csv('/content/drive/MyDrive/Apple_products.csv')
apple_df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Five Stars,perfect,[],B00NQHZ2RU,B00NQHZ2RU,AGYCCDOVYOOUDZOETUJN4U5KAGIA,1486140013000,0,True
1,5.0,LOVE this iPhone 6,So far extremely happy with the iPhone 6. Thi...,[],B00NQGOZCY,B00NQGOZCY,AHGSUXT56Q3XVYD6J3YBZMBOCXHA,1495430457000,2,True
2,1.0,Malfunction a lot,I would not recommend buying any electronics f...,[],B07234GKN5,B07234GKN5,AFV4KKH356TLSZEHI3UBHN4SYSJQ,1582741702541,2,True
3,1.0,Battery is done for.,Battery is trashed essentially making the phon...,[],B07CQNF813,B07CQNF813,AFTPWGVZUDMJE5UUADHHZBQK4AIQ,1581536080691,0,True
4,5.0,Brand new phone,This is literally a brand new phone for a used...,[],B09JFN8K6T,B09JFN8K6T,AFBJCNQY5YUKIKOZT4OB2EQ7JWXA,1671675512431,3,True


In [9]:
reviews = apple_df['text'].to_list()

In [10]:
reviews

['perfect',
 "So far extremely happy with the iPhone 6.  This works perfectly for what it was purchased for and nice to have the larger storage.  Wasn't sure what iPhone to get, but very happy with this purchase and would highly recommend to anyone looking to get one.  While it isn't the latest model, it still has many of the extra added features advertised by Apple.  no regrets and happy to have this one",
 'I would not recommend buying any electronics from this company; my phone came defected.',
 'Battery is trashed essentially making the phone unusable.  Complete waste of money.',
 'This is literally a brand new phone for a used phone price. I could not find a single scratch on this phone. I am so happy with this purchase. Saved a ton of money too.',
 'My daughter loves this phone and has had no problems with it.',
 'Works well!',
 'This iPhone 8+ was no good out the box.  Screen Res was horrible, volume no good.  Will be returning it.',
 'Came in mint condition!! 100% battery life.

In [11]:
prompt ="""Recognize all aspect terms with their corresponding sentiment polarity in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or features of the product/service. Determine the sentiment polarity from the options [\"positive\", \"negative\", \"neutral\"]. Answer in the format [\"aspect\", \"sentiment\"] without any explanation. If no aspect term exists, then only answer \"[]\"."
          """

In [13]:
aspects = []
for review in reviews:
  prompt_text = prompt+"'''"+review+"'''"
  output = llama3_70b(prompt_text)
  aspects.append(output)

In [15]:
len(aspects)

562

In [14]:
aspects

['[["perfect", "positive"]]',
 '[["iPhone 6", "positive"], ["storage", "positive"], ["purchase", "positive"], ["features", "positive"]]',
 '["electronics", "negative"], ["phone", "negative"], ["company", "negative"]',
 '["Battery", "negative"], ["phone", "negative"], ["money", "negative"]',
 '[["phone", "positive"], ["price", "positive"], ["purchase", "positive"]]',
 '[["phone", "positive"]]',
 '[["product", "positive"]]',
 '[["iPhone 8+", "negative"], ["Screen Res", "negative"], ["volume", "negative"]]',
 '["condition", "positive"], ["battery life", "positive"], ["scratches", "negative"], ["Brand", "neutral"]',
 '[["Apple Store", "neutral"], ["Genius", "neutral"], ["phone", "positive"]]',
 '["phone", "positive"], ["condition", "positive"], ["scratches", "negative"]',
 '[["price", "positive"], ["shipping", "positive"], ["product", "positive"], ["quality", "positive"], ["value", "positive"]]',
 '[["phone", "positive"], ["size", "positive"]]',
 '[["phone plan", "negative"], ["phone", "po

In [16]:
apple_df['aspects'] = aspects

In [17]:
apple_df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,aspects
0,5.0,Five Stars,perfect,[],B00NQHZ2RU,B00NQHZ2RU,AGYCCDOVYOOUDZOETUJN4U5KAGIA,1486140013000,0,True,"[[""perfect"", ""positive""]]"
1,5.0,LOVE this iPhone 6,So far extremely happy with the iPhone 6. Thi...,[],B00NQGOZCY,B00NQGOZCY,AHGSUXT56Q3XVYD6J3YBZMBOCXHA,1495430457000,2,True,"[[""iPhone 6"", ""positive""], [""storage"", ""positi..."
2,1.0,Malfunction a lot,I would not recommend buying any electronics f...,[],B07234GKN5,B07234GKN5,AFV4KKH356TLSZEHI3UBHN4SYSJQ,1582741702541,2,True,"[""electronics"", ""negative""], [""phone"", ""negati..."
3,1.0,Battery is done for.,Battery is trashed essentially making the phon...,[],B07CQNF813,B07CQNF813,AFTPWGVZUDMJE5UUADHHZBQK4AIQ,1581536080691,0,True,"[""Battery"", ""negative""], [""phone"", ""negative""]..."
4,5.0,Brand new phone,This is literally a brand new phone for a used...,[],B09JFN8K6T,B09JFN8K6T,AFBJCNQY5YUKIKOZT4OB2EQ7JWXA,1671675512431,3,True,"[[""phone"", ""positive""], [""price"", ""positive""],..."


In [18]:
cols_to_drop = ['title','images','user_id','helpful_vote','verified_purchase']
apple_df = apple_df.drop(columns=cols_to_drop)
apple_df.head()

,rating,text,asin,parent_asin,timestamp,aspects
0,5.0,perfect,B00NQHZ2RU,B00NQHZ2RU,1486140013000,"[[""perfect"", ""positive""]]"
1,5.0,So far extremely happy with the iPhone 6. Thi...,B00NQGOZCY,B00NQGOZCY,1495430457000,"[[""iPhone 6"", ""positive""], [""storage"", ""positi..."
2,1.0,I would not recommend buying any electronics f...,B07234GKN5,B07234GKN5,1582741702541,"[""electronics"", ""negative""], [""phone"", ""negati..."
3,1.0,Battery is trashed essentially making the phon...,B07CQNF813,B07CQNF813,1581536080691,"[""Battery"", ""negative""], [""phone"", ""negative""]..."
4,5.0,This is literally a brand new phone for a used...,B09JFN8K6T,B09JFN8K6T,1671675512431,"[[""phone"", ""positive""], [""price"", ""positive""],..."


In [19]:
apple_df.to_csv('/content/drive/MyDrive/Apple_products_with_aspects.csv')